In [ ]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

In [ ]:
await Import.FromFile("../Files/TransactionalData/NominalCashflows_CH_2020_12.csv").WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [ ]:
//Import.DefineFormat(ImportFormats.YieldCurve, async (options, dataSet) => {
    Activity.Start();
    var workspace =  Workspace.CreateNew();
    workspace.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>().DisableInitialization<YieldCurve>());
    var parsingLog = await Import.FromFile("../Files/Parameters/YieldCurveScenario.csv").WithType<YieldCurve>().WithTarget(workspace).ExecuteAsync();  // TODO: FromDataSet(dataSet)
    if(parsingLog.Errors.Any()) return parsingLog; 

    var partitions = await DataSource.Query<PartitionByReportingNodeAndPeriod>().ToArrayAsync();
    var newYieldCurves = await workspace.Query<YieldCurve>().ToArrayAsync();
    var savedYieldCurves = await DataSource.Query<YieldCurve>().ToArrayAsync();
    newYieldCurves = newYieldCurves.Except(savedYieldCurves, YieldCurveComparer.Instance()).ToArray();
    var reportingNodeByPartition = await DataSource.Query<PartitionByReportingNode>().ToDictionaryAsync(x => x.Id, x => x.ReportingNode);
    var inactiveDataNodesByPeriodAndScenario = DataSource.Query<DataNodeState>()
                                .Where(x => x.State == State.Inactive)
                                .GroupBy(x => new Tuple<int,int,string> (x.Year, x.Month, x.Scenario).ToValueTuple())
                                .Select(x => new{ Year = x.Key.Item1, 
                                                  Month = x.Key.Item2, 
                                                  Scenario = x.Key.Item3,
                                                  DataNodes = x.Select(y => y.DataNode)});

    var dataNodesByCurrencyAndReportingNode = DataSource.Query<DataNode>()
                                                .GroupBy(x => new Tuple<string, Guid>(x.FunctionalCurrency, x.Partition).ToValueTuple())
                                                .Select(x => new { Currency = x.Key.Item1, 
                                                                   ReportingNode = reportingNodeByPartition[x.Key.Item2], 
                                                                   DataNodes = x.Select(y => y.SystemName)});

    var updatableDataNodesByArgs = newYieldCurves.Join(dataNodesByCurrencyAndReportingNode, 
                                        yc => yc.Currency, dn => dn.Currency, 
                                        (yc, dn) => new { Args = new ImportArgs( dn.ReportingNode,
                                                                                 yc.Year, 
                                                                                 yc.Month,
                                                                                 Periodicity.Quarterly,
                                                                                 yc.Scenario,
                                                                                 ImportFormats.Cashflow), 
                                                          Currency = yc.Currency,
                                                          DataNodes = dn.DataNodes }).ToArray();
    
    foreach (var groupOfArgs in updatableDataNodesByArgs.GroupBy(x => new {ReportingNode = x.Args.ReportingNode, Scenario = x.Args.Scenario, Currency = x.Currency})) {
        var item = groupOfArgs.OrderBy(x => x.Args.Year).ThenBy(x => x.Args.Month).First();
        var newArgs = partitions.Where(x => x.Scenario == item.Args.Scenario && x.ReportingNode == item.Args.ReportingNode &&
                                            (x.Year >= item.Args.Year || (x.Year == item.Args.Year && x.Month >= item.Args.Month)))
                                .Select(x => new ImportArgs(x.ReportingNode, x.Year, x.Month, Periodicity.Quarterly, x.Scenario, ImportFormats.Cashflow)).ToArray();
        updatableDataNodesByArgs = updatableDataNodesByArgs.Concat(newArgs.Select(x => new { Args = x, Currency = item.Currency, DataNodes = item.DataNodes }).ToArray()).ToArray();
    }
    
    // filter for active data nodes by period and scenario
    foreach(var inactiveDataNodes in inactiveDataNodesByPeriodAndScenario) {
        updatableDataNodesByArgs.Where(x => x.Args.Year == inactiveDataNodes.Year && x.Args.Month == inactiveDataNodes.Month && x.Args.Scenario == inactiveDataNodes.Scenario)
                                .Select(x => new {Args = x.Args, Currency = x.Currency, DataNodes = x.DataNodes.Except(inactiveDataNodes.DataNodes)}).ToArray().ToArray();
    }
                            
    updatableDataNodesByArgs = updatableDataNodesByArgs.OrderBy(x => x.Args.Year).ThenBy(x => x.Args.Month).ThenBy(x => x.Args.Scenario).ToArray();
    
    //foreach (var dataNodesInPartition in updatableDataNodesByPartition){
        var dataNodesInPartition = updatableDataNodesByArgs.First();
        await DataSource.UpdateAsync<PartitionByReportingNode>( new[] { new PartitionByReportingNode { 
                                   Id = (Guid)(await DataSource.Partition.GetKeyForInstanceAsync<PartitionByReportingNode>(dataNodesInPartition.Args)),
                                   ReportingNode = dataNodesInPartition.Args.ReportingNode, 
                                   Scenario = dataNodesInPartition.Args.Scenario } } );
                                    
        await DataSource.UpdateAsync<PartitionByReportingNodeAndPeriod>( new[]{ new PartitionByReportingNodeAndPeriod { 
                                   Id = (Guid)(await DataSource.Partition.GetKeyForInstanceAsync<PartitionByReportingNodeAndPeriod>(dataNodesInPartition.Args)),
                                   Year = dataNodesInPartition.Args.Year,
                                   Month = dataNodesInPartition.Args.Month,
                                   ReportingNode = dataNodesInPartition.Args.ReportingNode, 
                                   Scenario = dataNodesInPartition.Args.Scenario } } );
        await DataSource.CommitAsync();
        // var storage = new ImportStorage(dataNodesInPartition.Args, DataSource, workspace);
        //await storage.InitializeAsync();
        // debug ImportStorage
        var args = dataNodesInPartition.Args;
        var CurrentReportingPeriod = (args.Year, args.Month);
        var PreviousReportingPeriod = (args.Year - 1, MonthInAYear); // YTD Logic
        //Dimensions
        var estimateTypes = (await workspace.Query<EstimateType>().ToArrayAsync());
        
        var EstimateTypeDimension     = estimateTypes.ToDictionary(x => x.SystemName);
        var AmountTypeDimension       = (await workspace.Query<AmountType>().ToArrayAsync()).ToDictionary(x => x.SystemName);
        var NoveltyDimension          = (await workspace.Query<Novelty>().ToArrayAsync()).ToDictionary(x => x.SystemName);
        
        var ExperienceAdjustEstimateTypeMapping = new Dictionary<string,string>{{EstimateTypes.A, EstimateTypes.APA}, {EstimateTypes.AA, EstimateTypes.AAPA}, {EstimateTypes.OA, EstimateTypes.OAPA}}; //TODO move this logic
        
        //Hierarchy Cache
        //hierarchyCache.Initialize<AmountType>();
        
        //EstimateType to load and to update
        var EstimateTypesByImportFormat = new InputSource[] { InputSource.Opening, InputSource.Actual, InputSource.Cashflow,}
                                        .ToDictionary(x => x.ToString(), 
                                                      x => estimateTypes
                                                      .Where(et => et.InputSource.Contains(x))
                                                      .Select(et => et.SystemName)
                                                      .ToHashSet());
        
        //ProjectionConfiguration : Current Period + projection for every Quarter End for current Year and next Years as in projectionConfiguration.csv
        var ProjectionConfiguration = (await workspace.Query<ProjectionConfiguration>()
                                               .Where(x => x.Shift > 0 || x.TimeStep == args.Month || (x.TimeStep > args.Month && x.TimeStep % 3 == 0))
                                               .OrderBy(x => x.Shift).ThenBy(x => x.TimeStep).ToArrayAsync());
        
        //Get Partitions
        var PartitionByRn = (await DataSource.Query<PartitionByReportingNode>().Where(p => p.ReportingNode == args.ReportingNode &&
                                                                                     p.Scenario == args.Scenario).ToArrayAsync()).Single().Id;
        var TargetPartition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == args.ReportingNode &&
                                                                                                   p.Year == CurrentReportingPeriod.Year &&
                                                                                                   p.Month == CurrentReportingPeriod.Month &&
                                                                                                   p.Scenario == args.Scenario).ToArrayAsync()).Single().Id;
        //TODO: what should happen when we import a YC for a scenario but the relative BE does not exist because it is not imported yet? 
        
        /*var DefaultPartition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == args.ReportingNode &&
                                                                                                    p.Year == CurrentReportingPeriod.Year &&
                                                                                                    p.Month == CurrentReportingPeriod.Month &&
                                                                                                    p.Scenario == null).ToArrayAsync()).Single().Id;
        if(Activity.HasErrors()) return Activity.Finish().Merge(parsingLog);

        var universe = Scopes.ForStorage(storage).ToScope<IModel>();
        var identities = universe.GetScopes<GetIdentities>(storage.DataNodesByImportScope[ImportScope.Primary]).SelectMany(s => s.Identities);
        var ivs = universe.GetScopes<ComputeAllScopes>(identities).SelectMany(x => x.CalculatedIfrsVariables);
        //calculatedIfrsVariables = calculatedIfrsVariables.Concat(ivs);
        if(Activity.HasErrors()) return Activity.Finish().Merge(parsingLog);
        await workspace.UpdateAsync<IfrsVariable>(ivs);
        await CommitToDatabase<IfrsVariable>(storage.TargetPartition, 
                                         snapshot : true, 
                                         filter : x => storage.EstimateTypesByImportFormat[ImportFormats.Cashflow].Contains(x.EstimateType) && 
                                                       storage.DataNodesByImportScope[ImportScope.Primary].Contains(x.DataNode));  
        await CommitToDatabase<RawVariable>(storage.TargetPartition, 
                                        snapshot : true,
                                        filter : x => storage.DataNodesByImportScope[ImportScope.Primary].Contains(x.DataNode));

    //}
    await DataSource.UpdateAsync<YieldCurve>( await Workspace.Query<YieldCurve>().ToArrayAsync() );
    await DataSource.CommitAsync();*/
    Activity.Finish()


In [ ]:
updatableDataNodesByArgs

In [ ]:
var temp = newYieldCurves.Except(savedYieldCurves, YieldCurveComparer.Instance());
temp

In [ ]:
temp.Intersect(savedYieldCurves, YieldCurveComparer.Instance()).ToArray().Count()

In [ ]:
var temp = updatedYieldCurves.Intersect(savedYieldCurves, YieldCurveComparer.Instance()).ToArray();
YieldCurveComparer.Instance().Equals(updatedYieldCurves[0], savedYieldCurves[0])


In [ ]:
var x = new string[] {"A", "B", "C"};
var y = new string[]  {"B", "C"};
x.Except(y)

In [ ]:
var x = updatedYieldCurves[0];

In [ ]:
var y = savedYieldCurves[0];

In [ ]:
YieldCurveComparer.Instance().Equals(x,y)

In [ ]:
updatedYieldCurves.Except(savedYieldCurves, YieldCurveComparer.Instance())

In [ ]:
savedYieldCurves